In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
from sklearn.model_selection import train_test_split
import seaborn as sns

In [4]:
import statistics

In [2]:
from dtaidistance import dtw
from dtaidistance import dtw_ndim

In [5]:
real_test = pd.read_csv("data_168_12_test.csv")
real_traindata = pd.read_csv("data_168_12_train.csv")
timeGANdata = pd.read_csv("timegan_32_50_2000.csv")
deepEchodata = pd.read_csv("deepEcho_168_2000epoch.csv")
ehrMGANdata = pd.read_csv("ehrMGAN_168_30_400_400f.csv")

In [9]:
# select sample equal to test dataset size 
def getsample(data, n=0.4):
    trainids, testids = train_test_split(list(set(data["hadm_id"])), test_size=n, random_state=499)
    return data[data["hadm_id"].isin(testids)]

real_train = getsample(real_traindata)
timeGAN = getsample(timeGANdata)
deepEcho = getsample(deepEchodata)
ehrMGAN = getsample(ehrMGANdata)


In [12]:
timeGAN

,hadm_id,hour,ABG,Insulin,Calcium,Blood Products,Antibiotics,Electrolytes,Glucose,Discharged,CBC,Urinalysis
0,0,0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
1,0,12,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
2,0,24,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,0,36,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,0,48,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7401,528,108,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
7402,528,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7403,528,132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7404,528,144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
timeGAN.drop(["hadm_id", "hour"], axis=1)

,ABG,Insulin,Calcium,Blood Products,Antibiotics,Electrolytes,Glucose,Discharged,CBC,Urinalysis
0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
1,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
2,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7401,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
7402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
def calcDTW2(data1, data2):
    data1ids = list(set(data1["hadm_id"]))
    data2ids = list(set(data2["hadm_id"]))
    dtwdic = {}
    for i in data1ids:
        seq1 = data1[data1["hadm_id"]==i]
        seq1 = seq1.drop(["hadm_id", "hour"], axis=1)
        seq1 = seq1.to_numpy()
        dist = []
        for j in data2ids:
            seq2 = data2[data2["hadm_id"]==j]
            seq2 = seq2.drop(["hadm_id", "hour"], axis=1)
            seq2 = seq2.to_numpy()
            temp = dtw_ndim.distance(seq1, seq2)
            
            dist.append(temp)
        dtwdic[i] = dist
        
    matrix = pd.DataFrame.from_dict(dtwdic, orient='index', columns=data2ids)
    return matrix



In [29]:
realmatrix = calcDTW2(real_test, real_train)
sns.heatmap(realmatrix)

KeyboardInterrupt: 

In [63]:
def calcDTW(data1, data2, columns):
    data1ids = list(set(data1["hadm_id"]))
    data2ids = list(set(data2["hadm_id"]))
    distancesmean,distancesmedian, distancessdv = {}, {}, {}
    for c in columns:
        distances = []
        for i in data1ids:
            data1seq = data1[data1["hadm_id"]==i][c].tolist()
            for j in data2ids:
                data2seq = data2[data2["hadm_id"]==j][c].tolist()
                distances.append(dtw.distance(data1seq, data2seq))
                
        distancesmean[c] = statistics.mean(distances)   
        distancesmedian[c] = statistics.median(distances)
        distancessdv[c] = statistics.stdev(distances) 
        print("done column ", c)
        
    return distancesmean,distancesmedian, distancessdv
            
    

In [10]:
columns = list(real_test.columns)[2:]

In [69]:
distancesmean,distancesmedian, distancessdv = calcDTW(real_test, real_train, columns)

done column  ABG
done column  Insulin
done column  Calcium
done column  Blood Products
done column  Antibiotics
done column  Electrolytes
done column  Glucose
done column  Discharged
done column  CBC
done column  Urinalysis


In [71]:
distancesmeantime,distancesmediantime, distancessdvtime = calcDTW(real_test, timeGAN , columns)

done column  ABG
done column  Insulin
done column  Calcium
done column  Blood Products
done column  Antibiotics
done column  Electrolytes
done column  Glucose
done column  Discharged
done column  CBC
done column  Urinalysis


In [72]:
distancesmeandeep,distancesmediandeep, distancessdvdeep = calcDTW(real_test, deepEcho, columns)

done column  ABG
done column  Insulin
done column  Calcium
done column  Blood Products
done column  Antibiotics
done column  Electrolytes
done column  Glucose
done column  Discharged
done column  CBC
done column  Urinalysis


In [44]:
#distancesmeanehr,distancesmedianehr, distancessdvehr = calcDTW(real_test, ehrMGAN, columns)

done column  ABG
done column  Insulin
done column  Calcium
done column  Blood Products
done column  Antibiotics
done column  Electrolytes
done column  Glucose
done column  Discharged
done column  CBC
done column  Urinalysis


In [73]:
distmeandic = {}
distmeandic["real"]  = list(distancesmean.values()) + [statistics.mean(list(distancesmean.values()))]
distmeandic["deepEcho"]  = list(distancesmeantime.values()) + [statistics.mean(list(distancesmeantime.values()))]
distmeandic["timeGAN"]  = list(distancesmeandeep.values()) + [statistics.mean(list(distancesmeandeep.values()))]
distmeandic["EHR-M-GAN"]  = list(distancesmeanehr.values()) + [statistics.mean(list(distancesmeanehr.values()))]

In [74]:
distmediandic = {}
distmediandic["real"]  = list(distancesmedian.values()) + [statistics.median(list(distancesmedian.values()))]
distmediandic["deepEcho"]  = list(distancesmediantime.values()) + [statistics.median(list(distancesmediantime.values()))]
distmediandic["timeGAN"]  = list(distancesmediandeep.values()) + [statistics.median(list(distancesmediandeep.values()))]
distmediandic["EHR-M-GAN"]  = list(distancesmedianehr.values()) + [statistics.median(list(distancesmedianehr.values()))]


In [75]:
print("Mean DTW")
pd.DataFrame.from_dict(distmeandic, orient='index', columns=columns+["total avg"])

Mean DTW


,ABG,Insulin,Calcium,Blood Products,Antibiotics,Electrolytes,Glucose,Discharged,CBC,Urinalysis,total avg
real,0.737256,0.538817,0.702831,0.476065,0.716546,0.820927,0.874033,0.036021,0.796976,0.384604,0.608408
deepEcho,0.840858,0.577679,0.769376,0.728458,0.787475,0.798901,0.815174,0.579796,0.795632,0.463936,0.715728
timeGAN,0.786035,0.568687,0.765172,0.467522,0.777413,0.883609,0.865159,0.069783,0.844053,0.431796,0.645923
EHR-M-GAN,0.619117,0.280458,0.528024,0.200000,0.407706,1.378039,1.302057,1.000000,1.396034,0.370711,0.748215


In [76]:
print("Median DTW")
pd.DataFrame.from_dict(distmediandic, orient='index', columns=columns+["total avg"])

Median DTW


,ABG,Insulin,Calcium,Blood Products,Antibiotics,Electrolytes,Glucose,Discharged,CBC,Urinalysis,total avg
real,1.0,0.0,1.0,0.0,1.0,1.000000,1.000000,0.0,1.000000,0.0,1.00
deepEcho,1.0,0.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,0.0,1.00
timeGAN,1.0,0.0,1.0,0.0,1.0,1.000000,1.000000,0.0,1.000000,0.0,1.00
EHR-M-GAN,0.5,0.0,0.0,0.0,0.0,1.414214,1.414214,1.0,1.414214,0.0,0.25


Averaging for dynamic time warping is the problem of finding an average sequence for a set of sequences. NLAAF[16] is an exact method to average two sequences using DTW. For more than two sequences, the problem is related to the one of the multiple alignment and requires heuristics. DBA[17] is currently a reference method to average a set of sequences consistently with DTW. COMASA[18] efficiently randomizes the search for the average sequence, using DBA as a local optimization process.

In [34]:
corr = real_test.corr()
corr.style.background_gradient(cmap='coolwarm')

,hadm_id,hour,magnesium,lactate,creatinine,CBC,calcium,Urine,ABG,leukocytes,anion gap,pt,urea nitrogen,bicarbonate,Electrolytes
hadm_id,1.000000,0.000000,0.002094,0.003841,-0.000364,0.010883,0.001412,0.005678,0.008044,0.010708,-0.000533,0.008093,-0.000621,-0.000592,0.009495
hour,0.000000,1.000000,0.044060,-0.274942,-0.034242,-0.283652,-0.285233,-0.338219,-0.390897,-0.095284,-0.021249,-0.191416,-0.034133,-0.043323,-0.307391
magnesium,0.002094,0.044060,1.000000,0.046045,0.689364,0.471494,0.273764,0.404225,0.196065,-0.002351,0.682231,0.202290,0.685433,0.661467,0.506116
lactate,0.003841,-0.274942,0.046045,1.000000,0.036963,0.429084,0.579620,0.498129,0.556676,0.296133,0.042179,0.303014,0.035927,0.039808,0.458731
creatinine,-0.000364,-0.034242,0.689364,0.036963,1.000000,0.606341,0.348878,0.499109,0.287467,0.014166,0.931840,0.484295,0.997446,0.968355,0.584233
CBC,0.010883,-0.283652,0.471494,0.429084,0.606341,1.000000,0.544796,0.720143,0.606665,0.192093,0.578817,0.497991,0.607281,0.602175,0.821130
calcium,0.001412,-0.285233,0.273764,0.579620,0.348878,0.544796,1.000000,0.733948,0.713512,0.205306,0.327272,0.472724,0.349055,0.358123,0.698445
Urine,0.005678,-0.338219,0.404225,0.498129,0.499109,0.720143,0.733948,1.000000,0.839147,0.165431,0.497604,0.381082,0.499168,0.511137,0.839938
ABG,0.008044,-0.390897,0.196065,0.556676,0.287467,0.606665,0.713512,0.839147,1.000000,0.177681,0.266222,0.405521,0.288577,0.292754,0.704537
leukocytes,0.010708,-0.095284,-0.002351,0.296133,0.014166,0.192093,0.205306,0.165431,0.177681,1.000000,0.013136,0.359285,0.014286,0.011452,0.164528


In [35]:
corr = timegan_test.corr()
corr.style.background_gradient(cmap='coolwarm')

/Users/lilyfelstead/opt/anaconda3/lib/python3.9/site-packages/pandas/io/formats/style.py:3555: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/Users/lilyfelstead/opt/anaconda3/lib/python3.9/site-packages/pandas/io/formats/style.py:3556: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,hadm_id,hour,magnesium,lactate,creatinine,CBC,calcium,Urine,ABG,leukocytes,anion gap,pt,urea nitrogen,bicarbonate,Electrolytes
hadm_id,1.000000,0.000000,nan,nan,nan,-0.012555,-0.009736,-0.011313,-0.009207,nan,nan,nan,nan,nan,-0.010280
hour,0.000000,1.000000,nan,nan,nan,-0.248984,-0.144596,-0.359896,-0.399771,nan,nan,nan,nan,nan,-0.359606
magnesium,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
lactate,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
creatinine,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CBC,-0.012555,-0.248984,nan,nan,nan,1.000000,0.615877,0.708652,0.650102,nan,nan,nan,nan,nan,0.713244
calcium,-0.009736,-0.144596,nan,nan,nan,0.615877,1.000000,0.436443,0.400383,nan,nan,nan,nan,nan,0.439270
Urine,-0.011313,-0.359896,nan,nan,nan,0.708652,0.436443,1.000000,0.892368,nan,nan,nan,nan,nan,0.957395
ABG,-0.009207,-0.399771,nan,nan,nan,0.650102,0.400383,0.892368,1.000000,nan,nan,nan,nan,nan,0.911472
leukocytes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
# need to find a way to display this...
# could display either as scatter plot or heatmap
# heatmap probably makes the most sense 
# makes sense to cluster

In [77]:
from tslearn.clustering import TimeSeriesKMeans

In [ ]:
model = TimeSeriesKMeans(n_clusters=3, metric="dtw",
                         max_iter=10, random_state=seed)
model.fit(X_train)